# Weather Function Calling Streaming Example

## Launching the Server
To begin, you need to launch a server that will handle the requests. The following code block is used to launch the server.
After the server is running, you can interact with it using the client to get the weather information.

```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0
```
This starts the server on port `30000`. Once running, you can make requests to it from this notebook.

In [ ]:
from sglang.utils import execute_shell_command, wait_for_server

server_process = execute_shell_command(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0"
)
wait_for_server("http://localhost:30000")

Traceback (most recent call last):
  File "/home/misc/yangmin/miniconda3/envs/sglang_fork/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/misc/yangmin/miniconda3/envs/sglang_fork/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/misc/yangmin/fork/sglang/python/sglang/launch_server.py", line 6, in <module>
    from sglang.srt.server import launch_server
  File "/home/misc/yangmin/fork/sglang/python/sglang/srt/server.py", line 65, in <module>
    from sglang.srt.openai_api.adapter import (
  File "/home/misc/yangmin/fork/sglang/python/sglang/srt/openai_api/adapter.py", line 76, in <module>
    from sglang.srt.function_call_parser import FunctionCallParser
ModuleNotFoundError: No module named 'sglang.srt.function_call_parser'


## Define Tools for Function Call
Next, we'll define the tools for our function call. In this example, we define a function for getting the current weather in a specified location.

In [ ]:
from openai import OpenAI
import json

# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston today? Please respond with the format: Today's weather is :{function call result}",
    }
]

# Initialize OpenAI-like client
client = OpenAI(api_key="YOUR_API_KEY", base_url="http://0.0.0.0:30000/v1")
model_name = client.models.list().data[0].id

## Make Non-Streaming Request
We'll now test the non-streaming function call and print the result.

In [ ]:
# Non-streaming mode test
response_non_stream = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8,
    top_p=0.8,
    stream=False,  # Non-streaming
    tools=tools,
)
print("Non-stream response:")
print(response_non_stream)

## Make Streaming Request
Next, we will test the streaming function call. The response will come in chunks, and we will handle the stream and process the function calls.

In [ ]:
# Streaming mode test
print("Streaming response:")
response_stream = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8,
    top_p=0.8,
    stream=True,  # Enable streaming
    tools=tools,
)

# Handle streaming responses, combine different chunks
chunks = []
for chunk in response_stream:
    chunks.append(chunk)
    print(chunk)  # Optionally print each chunk to observe its content

# Parse and combine function call arguments
arguments = []
for chunk in chunks:
    choice = chunk.choices[0]
    delta = choice.delta
    if delta.tool_calls:
        tool_call = delta.tool_calls[0]
        if tool_call.function.name:
            print(f"Streamed function call name: {tool_call.function.name}")

        if tool_call.function.arguments:
            arguments.append(tool_call.function.arguments)
            print(f"Streamed function call arguments: {tool_call.function.arguments}")

# Combine all argument fragments
full_arguments = "".join(arguments)
print(f"Final streamed function call arguments: {full_arguments}")

## Simulate Tool Call
In this section, we'll simulate a call to the weather function and handle the arguments passed from the stream.

In [ ]:
# Define the actual function for getting current weather
def get_current_weather(location: str, unit: str):
    # Here you can integrate an actual weather API
    return f"The weather in {location} is 85 degrees {unit}. It is partly cloudy, with highs in the 90's."


# Simulate tool call
available_tools = {"get_current_weather": get_current_weather}

# Parse JSON arguments
try:
    call_data = json.loads(full_arguments)
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")
    call_data = {}

# Call the corresponding tool function
if "name" in messages[-1]["tool_calls"]:
    tool_name = messages[-1]["tool_calls"]["name"]
    if tool_name in available_tools:
        tool_to_call = available_tools[tool_name]
        result = tool_to_call(**call_data)
        print(f"Function call result: {result}")
        messages.append({"role": "tool", "content": result, "name": tool_name})
    else:
        print(f"Unknown tool name: {tool_name}")
else:
    print("Function call name not found.")

## Final Chat Completion
We will now perform the final chat completion using the simulated function call result.

In [ ]:
chat_completion_final = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0.8,
    top_p=0.8,
    stream=False,
    tools=tools,
)

print("\nFinal Chat Completion:")
print(chat_completion_final)

## Terminate Server
Once you are done, you can terminate the server process using the following code.

In [ ]:
terminate_process(server_process)